# 含并行连结的网络（GoogLeNet）

在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [1]。它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。在随后的几年里，研究人员对GoogLeNet进行了数次改进，本节将介绍这个模型系列的第一个版本。


## Inception 块

GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上一节介绍的NiN块相比，这个基础块在结构上更加复杂，如图5.8所示。

![Inception块的结构](../img/inception.svg)

由图5.8可以看出，Inception块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

Inception块中可以自定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [0]:
import d2ltorch as d2lt
import torch
from torch import nn, optim
from torchsummary import summary

class Inception(nn.Module):
    # c0为输入通道数, c1 - c4为每条线路里的层的输出通道数
    def __init__(self, c0, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        # 线路1，单1 x 1卷积层
        self.p1 = nn.Sequential(
            nn.Conv2d(c0, c1, kernel_size=1), nn.ReLU(inplace=True)
            
        )
        
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2 = nn.Sequential(
            nn.Conv2d(c0, c2[0], kernel_size=1), 
            nn.ReLU(inplace=True),
            nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1), 
            nn.ReLU(inplace=True)
        )
        
        # 线路3， 1 x 1卷积层后接5 x 5卷积层
        self.p3 = nn.Sequential(
            nn.Conv2d(c0, c3[0], kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2), nn.ReLU(inplace=True)
        )
        
        # 线路4， 3 x 3最大池化层后接1 x 1卷积层
        self.p4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(c0, c4, kernel_size=1), nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        x1 = self.p1(x)
        x2 = self.p2(x)
        x3 = self.p3(x)
        x4 = self.p4(x)
        return torch.cat((x1, x2, x3, x4), dim=1) # 在通道维上连结输出

## GoogLeNet模型

GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用步幅为2的$3\times 3$最大池化层来减小输出高宽。

In [0]:
class MyGoogLeNet(nn.Module):
    def __init__(self, in_channels, **kwargs):
        super(MyGoogLeNet, self).__init__(**kwargs)
        # 第一模块使用一个64通道的7 x 7卷积层。
        self.b1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 第二模块使用2个卷积层
        self.b2 = nn.Sequential(
            # 首先是64通道的1 x 1卷积层
            nn.Conv2d(64, 64, kernel_size=1), nn.ReLU(inplace=True),
            # 然后是将通道增大3倍的3 x 3卷积层。它对应Inception块中的第二条线路。
            nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 第三模块串联2个完整的Inception块。
        self.b3 = nn.Sequential(
            # 第一个Inception块的输出通道数为 64+128+32+32=256，
            # 其中4条线路的输出通道数比例为 64:128:32:32=2:4:1:1。
            # 其中第二、第三条线路先分别将输入通道数减小至 96/192=1/2 和 16/192=1/12 后，再接上第二层卷积层。
            Inception(192, 64, (96, 128), (16, 32), 32),
            # 第二个Inception块输出通道数增至 128+192+96+64=480，
            # 每条线路的输出通道数之比为 128:192:96:64 = 4:6:3:2。
            # 其中第二、第三条线路先分别将输入通道数减小至 128/256=1/2 和 32/256=1/8 。
            Inception(256, 128, (128, 192), (32, 96), 64),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 第四模块更加复杂。它串联了5个Inception块。
        # 每个Inception块的线路的通道数分配和第三模块中的类似，
        # 首先含3 x 3卷积层的第二条线路输出最多通道，其次是仅含1 x 1卷积层的第一条线路，
        # 之后是含5 x 5卷积层的第三条线路和含3 x 3最大池化层的第四条线路。
        # 其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。
        self.b4 = nn.Sequential(
            # 输出通道数 192+208+48+64=512
            Inception(480, 192, (96, 208), (16, 48), 64),
            # 输出通道数 160+224+64+64=512
            Inception(512, 160, (112, 224), (24, 64), 64),
            # 输出通道数 128+256+64+64=512
            Inception(512, 128, (128, 256), (24, 64), 64),
            # 输出通道数 112+288+64+64=528
            Inception(512, 112, (144, 288), (32, 64), 64),
            # 输出通道数 256+320+128+128=832
            Inception(528, 256, (160, 320), (32, 128), 128),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 第五模块有两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，
        # 只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，
        # 该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。
        self.b5 = nn.Sequential(
            # 输出通道数 256+320+128+128=832
            Inception(832, 256, (160, 320), (32, 128), 128),
            # 输出通道数 384+384+128+128=1024
            Inception(832, 384, (192, 384), (48, 128), 128),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # 最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。
        self.output = nn.Linear(1024*1*1, 10)
    
    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = x.reshape(-1, 1024*1*1)
        x = self.output(x)
        return x

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节里我们将输入的高和宽从224降到96来简化计算。下面演示各个模块之间的输出的形状变化。

In [3]:
net = MyGoogLeNet(in_channels=1)
summary(net, (1, 96, 96), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           3,200
              ReLU-2           [-1, 64, 48, 48]               0
         MaxPool2d-3           [-1, 64, 24, 24]               0
            Conv2d-4           [-1, 64, 24, 24]           4,160
              ReLU-5           [-1, 64, 24, 24]               0
            Conv2d-6          [-1, 192, 24, 24]         110,784
              ReLU-7          [-1, 192, 24, 24]               0
         MaxPool2d-8          [-1, 192, 12, 12]               0
            Conv2d-9           [-1, 64, 12, 12]          12,352
             ReLU-10           [-1, 64, 12, 12]               0
           Conv2d-11           [-1, 96, 12, 12]          18,528
             ReLU-12           [-1, 96, 12, 12]               0
           Conv2d-13          [-1, 128, 12, 12]         110,720
             ReLU-14          [-1, 128,

## 获取数据和训练模型

我们使用高和宽均为96像素的图像来训练GoogLeNet模型。训练使用的图像依然来自Fashion-MNIST数据集。

In [4]:
root = '~/dataset'
lr, num_epochs, batch_size, device = 0.1, 5, 128, d2lt.try_gpu()
    
d2lt.params_init(net, nn.init.xavier_uniform_)
optimizer = optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2lt.load_data_fashion_mnist(root, batch_size, resize=96)
d2lt.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device,
              num_epochs)

training on gpu
epoch 1, loss 0.0155, train acc 0.236, test acc 0.518, time 46.5 sec
epoch 2, loss 0.0063, train acc 0.683, test acc 0.764, time 47.7 sec
epoch 3, loss 0.0042, train acc 0.798, test acc 0.838, time 48.8 sec
epoch 4, loss 0.0032, train acc 0.846, test acc 0.857, time 49.8 sec
epoch 5, loss 0.0034, train acc 0.843, test acc 0.841, time 49.4 sec


## 小结

* Inception块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用$1\times 1$卷积层减少通道数从而降低模型复杂度。
* GoogLeNet将多个设计精细的Inception块和其他层串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。
* GoogLeNet和它的后继者们一度是ImageNet上最高效的模型之一：在类似的测试精度下，它们的计算复杂度往往更低。

## 练习

* GoogLeNet有数个后续版本。尝试实现并运行它们，然后观察实验结果。这些后续版本包括加入批量归一化层（下一节将介绍）[2]、对Inception块做调整 [3] 和加入残差连接（[“残差网络（ResNet）”](resnet.ipynb)一节将介绍）[4]。

* 对比AlexNet、VGG和NiN、GoogLeNet的模型参数尺寸。为什么后两个网络可以显著减小模型参数尺寸？




## 参考文献

[1] Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).

[2] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

[3] Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (2016). Rethinking the inception architecture for computer vision. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (pp. 2818-2826).

[4] Szegedy, C., Ioffe, S., Vanhoucke, V., & Alemi, A. A. (2017, February). Inception-v4, inception-resnet and the impact of residual connections on learning. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 4, p. 12).

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1662)

![](../img/qr_googlenet.svg)